In [64]:
import numpy as np

import pandas as pd

import datetime as dt


from joblib import dump, load

In [65]:
df = pd.read_csv("./san_francisco_crimes_data_from_2018/sf_larceny_from_2018.csv")

In [66]:
dates = pd.read_excel(r"C:\Users\LENOVO\Downloads\Date 2019-2030.xlsx")

In [67]:
dates.head(5)

,Date
0,2019-01-01
1,2019-01-02
2,2019-01-03
3,2019-01-04
4,2019-01-05


In [68]:
dates = pd.DataFrame(dates.iloc[::1, 0].values, columns = ['Date'])

In [69]:
dates['Date'] = dates.Date.apply(lambda x: dt.datetime.strftime(x, "%Y-%m-%d"))

In [70]:
df.head(5)

,incident_datetime,incident_date,incident_time,incident_hour,incident_month,incident_year,incident_day_of_week,report_datetime,incident_id,incident_category,incident_subcategory,resolution,police_district,latitude,longitude
0,2021-07-25 00:00:00,2021-07-25,00:00,0,7,2021,Sunday,2021-07-25 13:41:00,1057189,Larceny Theft,Other,Open or Active,Southern,NaN,NaN
1,2022-06-28 14:00:00,2022-06-28,14:00,14,6,2022,Tuesday,2022-06-28 15:09:00,1165462,Larceny Theft,From Vehicle,Open or Active,Richmond,NaN,NaN
2,2022-08-07 22:00:00,2022-08-07,22:00,22,8,2022,Sunday,2022-08-16 15:26:00,1182772,Larceny Theft,Other,Open or Active,Richmond,NaN,NaN
3,2022-06-23 00:00:00,2022-06-23,00:00,0,6,2022,Thursday,2022-06-28 00:03:00,1165504,Larceny Theft,From Vehicle,Open or Active,Mission,NaN,NaN
4,2021-09-28 20:00:00,2021-09-28,20:00,20,9,2021,Tuesday,2021-09-30 09:41:00,1076413,Larceny Theft,From Vehicle,Open or Active,Central,NaN,NaN


In [72]:
merged_df = pd.merge(dates, df, how = "outer", left_on = dates.columns.values.tolist()[0], right_on = "incident_date")

In [74]:
merged_df.head(5)

,Date,incident_datetime,incident_date,incident_time,incident_hour,incident_month,incident_year,incident_day_of_week,report_datetime,incident_id,incident_category,incident_subcategory,resolution,police_district,latitude,longitude
0,2019-01-01,2019-01-01 04:03:00,2019-01-01,04:03,4.0,1.0,2019.0,Tuesday,2021-08-02 15:30:00,1057236.0,Larceny Theft,Other,Open or Active,Taraval,37.763515,-122.478191
1,2019-01-01,2019-01-01 01:00:00,2019-01-01,01:00,1.0,1.0,2019.0,Tuesday,2021-08-10 15:22:00,1059687.0,Larceny Theft,Other,Open or Active,Mission,37.753025,-122.432024
2,2019-01-01,2019-01-01 00:00:00,2019-01-01,00:00,0.0,1.0,2019.0,Tuesday,2022-01-12 16:04:00,1111226.0,Larceny Theft,From Building,Open or Active,Richmond,37.778897,-122.468426
3,2019-01-01,2019-01-01 18:30:00,2019-01-01,18:30,18.0,1.0,2019.0,Tuesday,2019-06-02 19:48:00,807450.0,Larceny Theft,Other,Open or Active,Ingleside,37.720482,-122.450103
4,2019-01-01,2019-01-01 23:00:00,2019-01-01,23:00,23.0,1.0,2019.0,Tuesday,2019-01-02 08:55:00,755267.0,Larceny Theft,From Vehicle,Open or Active,Ingleside,37.733088,-122.444320


In [79]:
daily_df = merged_df.groupby(['Date', 'incident_date', 'incident_subcategory']).agg({"incident_id": pd.Series.count}).reset_index()

In [86]:
daily_df['Year'] = daily_df['Date'].apply(lambda x: dt.datetime.strftime(dt.datetime.strptime(x, "%Y-%m-%d"), "%Y"))

In [93]:
max_daily_df = daily_df.groupby(['incident_subcategory', 'Year']).agg({"incident_id": pd.Series.max}).reset_index()

In [94]:
min_daily_df = daily_df.groupby(['incident_subcategory', 'Year']).agg({"incident_id": pd.Series.min}).reset_index()

In [96]:
min_daily_df = min_daily_df.rename({"incident_id": "min_incident_id"}, axis = 1)

max_daily_df = max_daily_df.rename({"incident_id": "max_incident_id"}, axis = 1)

In [97]:
pd.concat([max_daily_df, pd.DataFrame(min_daily_df.min_incident_id)], axis = 1)

,incident_subcategory,Year,max_incident_id,min_incident_id
0,Auto Parts,2019,4,1
1,Auto Parts,2020,7,1
2,Auto Parts,2021,8,1
3,Auto Parts,2022,14,1
4,Auto Parts,2023,8,1
5,Bicycle,2019,7,1
6,Bicycle,2020,8,1
7,Bicycle,2021,6,1
8,Bicycle,2022,5,1
9,Bicycle,2023,3,1


In [101]:
incident_subcategories = daily_df.incident_subcategory.unique()

incident_subcategories

array(['Bicycle', 'From Building', 'From Vehicle', 'Other', 'Pickpocket',
       'Shoplifting', 'Auto Parts', 'Purse Snatch'], dtype=object)

In [104]:
daily_df.to_csv("./san_francisco_crimes_data_from_2018/daily_lar_incidents.csv", index = False)